In [ ]:
import pymc

# Define the variables needed for the routine, with their prior distributions
alpha = pymc.Uniform('alpha', -100, 100)

@pymc.stochastic(observed=False)
def beta(value=0):
    return -1.5 * np.log(1 + value ** 2)

@pymc.stochastic(observed=False)
def sigma(value=1):
    return -np.log(abs(value))

# Define the form of the model and likelihood
@pymc.deterministic
def y_model(x=xdata, alpha=alpha, beta=beta):
    return alpha + beta * x

@pymc.observed
def y(y_model=y_model, sigma=sigma, value=ydata):
    return pymc.distributions.noncentral_t_like(value, y_model, 1. / sigma ** 2, 1)


# package the full model in a dictionary
model1 = dict(alpha=alpha, beta=beta, sigma=sigma,
              y_model=y_model, y=y)

S = pymc.MCMC(model1)
S.sample(iter=100000, burn=50000)

In [ ]:
y_min = S.stats()['mu']['quantiles'][2.5]
y_max = S.stats()['mu']['quantiles'][97.5]
y_fit = S.stats()['mu']['mean']